In [1]:
!pip install datasets
!pip install nltk
from datasets import load_dataset
import random
import re
import nltk
from nltk.corpus import wordnet as wn
import copy
import datasets
import copy
from datasets import Dataset, load_from_disk
#from huggingface_hub import HfApi

nltk.download('wordnet')
random.seed(0)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 require

[nltk_data] Downloading package wordnet to /root/nltk_data...


Dataset loading and (partial) analysis.

In [2]:
#Load training dataset
sst2_dataset = load_dataset("tommasobonomo/sem_augmented_fever_nli")  #['id', 'premise', 'hypothesis', 'label', 'wsd', 'srl']
sst2_dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/51086 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2288 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2287 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'premise', 'hypothesis', 'label', 'wsd', 'srl'],
        num_rows: 51086
    })
    validation: Dataset({
        features: ['id', 'premise', 'hypothesis', 'label', 'wsd', 'srl'],
        num_rows: 2288
    })
    test: Dataset({
        features: ['id', 'premise', 'hypothesis', 'label', 'wsd', 'srl'],
        num_rows: 2287
    })
})

In [3]:
#Part of the dataset observations performed

"""for i in sst2_dataset['train'][0]['wsd']['premise']:
  print(i)"""

"""for i in sst2_dataset ['train'][0]['srl']['premise']['tokens']:
  print(i)"""

"""for i in sst2_dataset ['train'][2]['srl']['premise']['annotations']:
  print(i)"""

"for i in sst2_dataset ['train'][2]['srl']['premise']['annotations']:\n  print(i)"

In [4]:
#Analysis to find the most common verbs, useful for some alterations

"""from collections import Counter

verbs = []
for sample in sst2_dataset['train']:
  srl_annotations = sample['srl']['premise']['annotations']
  srl_tokens = sample['srl']['premise']['tokens']
  for sentence in srl_annotations:
    verb_index = sentence['tokenIndex']
    verb = srl_tokens[verb_index]['rawText']
    verbs.append(verb)

verbs_counter = Counter(verbs)
most_common_verbs = verbs_counter.most_common()[:200]
for v in most_common_verbs:
print(v)"""

"from collections import Counter\n\nverbs = []\nfor sample in sst2_dataset['train']:\n  srl_annotations = sample['srl']['premise']['annotations']\n  srl_tokens = sample['srl']['premise']['tokens']\n  for sentence in srl_annotations:\n    verb_index = sentence['tokenIndex']\n    verb = srl_tokens[verb_index]['rawText']\n    verbs.append(verb)\n\nverbs_counter = Counter(verbs)\nmost_common_verbs = verbs_counter.most_common()[:200]\nfor v in most_common_verbs:\nprint(v)"

Let's define some (4) augmentation function...


In [5]:
#SYNONYM_ALTERATION

#Replaces a word with a synonym.
#Uses wordnet and wsd information.
def synonym_alteration(sample, to_alter):

  altered = False
  if to_alter != 'hypothesis' and to_alter != 'premise':
    print("ERROR: to_alter must be 'hypothesis' or 'premise'.")
    return altered

  wsd = sample['wsd'][to_alter]
  new_sentence = ""

  for i in range(len(wsd)):
    w = wsd[i]
    #print("WSD INFO: " + w)
    synset_id = w['nltkSynset']
    pos = w['pos']
    if synset_id != "O" and (pos == "ADJ" or pos == "NOUN" or pos == "ADV"):  #We only replace words that (mostly) don't need to be declined, i.e. adjectives, nouns and adverbs.
      synset = wn.synset(synset_id)
      #print("ID: " + synset_id)
      lemmas = synset.lemmas()
      #print("LEMMAS: " + str(lemmas))
      selected_lemma = lemmas[random.randint(0, len(lemmas)-1)] #We select a random synonym.
      new_word = str(selected_lemma.name())
      if new_word != w['text']:         #If the random selected lemma is different from the lemma we wanted to substitue...
        new_sentence += new_word + " "  #...we substitute the word and stop iterating. No more words will be substituted, this should mitigate the risk of poisoning
        altered = True
        break
    else:                             #Else...
      new_sentence += w['text'] + " " #...it's the occasion to try substituting one of the next words.

  for j in range(i+1, len(wsd)):        #Let's copy the final part of the sentence
    new_sentence += wsd[j]['text'] + " "
  new_sentence = new_sentence[:-1]

  sample[to_alter] = new_sentence

  return altered

In [6]:
#HYPERNIM_ALTERATION

#Replaces a word with a hypernym.
#Uses wordnet and wsd information.
def hypernym_alteration(sample, to_alter):

  altered = False
  if to_alter != 'hypothesis' and to_alter != 'premise':
    print("ERROR: to_alter must be 'hypothesis' or 'premise'.")
    return altered

  wsd = sample['wsd'][to_alter]
  new_sentence = ""

  for i in range(len(wsd)):
    w = wsd[i]
    #print("WSD INFO: " + w)
    synset_id = w['nltkSynset']
    pos = w['pos']
    if synset_id != "O" and (pos == "ADJ" or pos == "NOUN" or pos == "ADV"):  #We only replace words that (mostly) don't need to be declined, i.e. adjectives, nouns and adverbs.
      synset = wn.synset(synset_id)
      hypernyms = synset.hypernyms()
      #print("ID: " + synset_id)
      lemmas = []
      for h in hypernyms:
        lemmas = h.lemmas()
      #print("LEMMAS: " + str(lemmas))
      if len(lemmas) == 0:
        continue
      selected_lemma = lemmas[random.randint(0, len(lemmas)-1)] #We select a random synonym.
      new_word = str(selected_lemma.name())
      if new_word != w['text']:         #If the random selected lemma is different from the lemma we wanted to substitue...
        new_sentence += new_word + " "  #...we substitute the word and stop iterating. No more words will be substituted, this should mitigate the risk of poisoning
        altered = True
        break
    else:                             #Else...
      new_sentence += w['text'] + " " #...it's the occasion to try substituting one of the next words.

  for j in range(i+1, len(wsd)):        #Let's copy the final part of the sentence
    new_sentence += wsd[j]['text'] + " "
  new_sentence = new_sentence[:-1]

  sample[to_alter] = new_sentence

  return altered

In [7]:
#BIRTH_DATE_QUERY

#Auxiliary function that looks for birth dates.
def find_date(sentence):
  pattern = "([\S]+) \( born (January|February|March|April|May|June|July|August|September|October|November|December) ([0-9]{1,2}) , ([0-9]{4}) \)"
  find = re.search(pattern, sentence)
  if find == None:
    return None
  else:
    name, month, day, year = find.groups()
    #print("NAME: " + name + " MONTH: " + month + " DAY: " + day + " YEAR: " + year)
    return [name, month, day, year]

#Generates a new hypothesis that affirms something related to the birth date and properly sets the label
#Uses regex
#Possible generated sentences:
#X was born exactly Y years before Z
#X was born exactly Y years after Z
#X turned Y in Z
def birth_date_query(sample):
  altered = False
  premise = sample['premise']
  date = find_date(premise)
  if date == None:
    return altered

  altered = True
  delta = random.randint(1,100)
  mode = random.randint(0,5)

  if mode == 0:
    new_hypothesis = date[0] + " was born exactly " + str(delta) + " years after " + date[2] + " " + date[1] + " " + str(int(date[3])-delta) +" ."
    sample['label'] = 'ENTAILMENT'
  elif mode == 1:
    new_hypothesis = date[0] + " was born exactly " + str(delta) + " years before " + date[2] + " " + date[1] + " " + str(int(date[3])+delta) +" ."
    sample['label'] = 'ENTAILMENT'
  elif mode == 2:
    new_hypothesis = date[0] + " turned " + str(delta) + " in " + str(int(date[3])+delta) +" ."
    sample['label'] = 'ENTAILMENT'
  elif mode == 3:
    other_delta = random.choice([i for i in range(1,100) if i != delta])
    new_hypothesis = date[0] + " was born exactly " + str(other_delta) + " years before " + date[2] + " " + date[1] + " " + str(int(date[3])+delta) +" ."
    sample['label'] = 'CONTRADICTION'
  elif mode == 4:
    other_delta = random.choice([i for i in range(1,100) if i != delta])
    new_hypothesis = date[0] + " was born exactly " + str(other_delta) + " years after " + date[2] + " " + date[1] + " " + str(int(date[3])-delta) +" ."
    sample['label'] = 'CONTRADICTION'
  elif mode == 5:
    other_delta = random.choice([i for i in range(1,100) if i != delta])
    new_hypothesis = date[0] + " turned " + str(other_delta) + " in " + str(int(date[3])+delta) +" ."
    sample['label'] = 'CONTRADICTION'

  sample['hypothesis'] = new_hypothesis
  return altered

In [8]:
#FACT_CHECKING_QUERY

#Auxiliary function that finds the main topic of the premise. Hopefully this is the subject of all the sentences in the premise but we will have some poisoning :(
def find_subject(sample):
  srl = sample['srl']['premise']['annotations']
  if len(srl) == 0: #Let's discard malformed samples.
    return None
  first_sentence = srl[0]
  roles = first_sentence['verbatlas']['roles']
  #print("PREMISE: " + sample['premise'])
  for w in roles:
    #print("ROLE: " + w['role'])
    #print("SPAN: " + str(w['span']))
    if w['role'] == 'Theme' and w['span'][0] == 0:  #If the subject (theme) of the first sentence starts spanning at 0, that is the topic we are looking for :)
      subject = ""
      for index in range(0, w['span'][1]):
        subject += sample['srl']['premise']['tokens'][index]['rawText'] + " "
      subject = subject[:-1]
      return subject
    else: #Else, the premise has probably the shape: "TOPIC . SENTENCES ABOUT TOPIC" (e.g. "Roman Atwood . He is a very famous youtuber...")
      premise = sample['premise']
      index = premise.find('.')
      subject = premise[:index-1]
      if len(subject) < 51: #For avoiding getting the whole first sentence in "malformed" samples let's set a limit to the lenght of the subject.
        return subject
  return None

#Creates a new hypothesis that adfirms or negates something known about the topic and properly sets the label.
#Uses srl information
def fact_checking_query(sample):

  first_subject = find_subject(sample)
  if first_subject == None: #If we don't find the topic we surrender
    return False

  srl_annotations = sample['srl']['premise']['annotations']
  srl_tokens = sample['srl']['premise']['tokens']
  annotation = srl_annotations[random.randint(0, len(srl_annotations)-1)] #Randomly pick a sentence to alter.
  verb_index = annotation['tokenIndex']
  verb = srl_tokens[verb_index]["rawText"]

  #For preventing poisoning, we only consider verbs that can be easily negated.
  #Beside the forms finishing with "ed" I manually selected common forms in the dataset.
  if verb[:-2] != "ed" and verb not in ["is", "was", "has", "won", "written", "are", "became", "stars", "began"]:
    return False

  attribute_span = [] #We reconstruct the Attribute
  for a in annotation['verbatlas']['roles']:
    if a['role'] == "Attribute":
      attribute_span = a['span']
      break
  if attribute_span == []:
    return False
  attribute = ""
  for i in range(attribute_span[0], attribute_span[1]):
    attribute = attribute + srl_tokens[i]['rawText'] + " "
  attribute = attribute[:-1]

  mode = random.randint(0,1)
  if mode == 0:
    new_hypothesis = first_subject + " " + verb + " not " + attribute + " ."
    sample['label'] = "CONTRADICTION"
  elif mode == 1:
    new_hypothesis = first_subject + " " + verb + " " + attribute + " ."
    sample['label'] = "ENTAILMENT"
  sample["hypothesis"] = new_hypothesis

  return True

In [9]:
#GROUP_INCLUSION_QUERY

#Creates a new hypothesis that asks about the inclusion of the topic in some category and properly sets the label.
#Uses srl information
def group_inclusion_query(sample):

  first_subject = find_subject(sample)
  if first_subject == None:
    return False

  srl_annotations = sample['srl']['premise']['annotations']
  srl_tokens = sample['srl']['premise']['tokens']
  annotation = srl_annotations[random.randint(0, len(srl_annotations)-1)]
  verb_index = annotation['tokenIndex']
  verb = srl_tokens[verb_index]["rawText"]

  if verb not in ["is", "was", "became"]:
    return False

  attribute_span = []
  for a in annotation['verbatlas']['roles']:
    if a['role'] == "Attribute":
      attribute_span = a['span']
      break
  if attribute_span == []:
    return False
  attribute = ""
  for i in range(attribute_span[0], attribute_span[1]):
    attribute = attribute + srl_tokens[i]['rawText'] + " "
  attribute = attribute[:-1]

  mode = random.randint(0,5)
  if mode == 0:
    new_hypothesis = "Considering all those who have ever been " + attribute + " there is not " + first_subject + " ."
    sample['label'] = "CONTRADICTION"
  elif mode == 1:
    new_hypothesis = "Considering all those who have ever been " + attribute + " there is " + first_subject + " ."
    sample['label'] = "ENTAILMENT"
  elif mode == 2:
    new_hypothesis = "In a list with all those that are " + attribute + " there is not " + first_subject + " ."
    sample['label'] = "CONTRADICTION"
  elif mode == 3:
    new_hypothesis = "In a list with all those that are " + attribute + " there is " + first_subject + " ."
    sample['label'] = "ENTAILMENT"
  elif mode == 4:
    amount = random.randint(5,100)
    new_hypothesis = "In a list with all those that are " + attribute + " there are " + str(amount) + " individuals ."
    sample['label'] = "NEUTRAL"
  elif mode == 5:
    amount = random.randint(5,100)
    new_hypothesis = "Considering all those who have ever been " + attribute + " there are " + str(amount) + " individuals ."
    sample['label'] = "NEUTRAL"

  sample["hypothesis"] = new_hypothesis
  return True

In [10]:
def random_join(sample1, sample2):
  sample1["hypothesis"] = sample2["hypothesis"]
  sample1["label"] = "NEUTRAL"
  return True

Some analysis to understand how many samples generate.

In [11]:
"""print(str(type(sst2_dataset['train'])))
print(str(len(sst2_dataset['train'])))
ent_counter = 0
con_counter = 0
neu_counter = 0
for sample in sst2_dataset['train']:
  if sample['label'] == "ENTAILMENT":
    ent_counter += 1
  if sample['label'] == "CONTRADICTION":
    con_counter += 1
  if sample['label'] == "NEUTRAL":
    neu_counter += 1
print("ENTAILMENT: " + str(ent_counter))
print("CONTRADICTION: " + str(con_counter))
print("NEUTRAL: " + str(neu_counter))"""

'print(str(type(sst2_dataset[\'train\'])))\nprint(str(len(sst2_dataset[\'train\'])))\nent_counter = 0\ncon_counter = 0\nneu_counter = 0\nfor sample in sst2_dataset[\'train\']:\n  if sample[\'label\'] == "ENTAILMENT":\n    ent_counter += 1\n  if sample[\'label\'] == "CONTRADICTION":\n    con_counter += 1\n  if sample[\'label\'] == "NEUTRAL":\n    neu_counter += 1\nprint("ENTAILMENT: " + str(ent_counter))\nprint("CONTRADICTION: " + str(con_counter))\nprint("NEUTRAL: " + str(neu_counter))'

Let's try all the functions...

In [12]:
print("SYNONIM ALTERATION")
counter = 0
while(counter < 1):
  original_sample = sst2_dataset['train'][random.randint(0,len(sst2_dataset['train'])-1)]
  sample = copy.deepcopy(original_sample)
  to_alter = random.randint(0,1)
  if to_alter == 0:
    to_alter = 'hypothesis'
  else:
    to_alter = 'premise'
  altered = synonym_alteration(sample, to_alter)
  if altered:
    counter += 1
    print("PRE: " + original_sample[to_alter])
    print("POST: " + sample[to_alter])

print("HYPERNYM ALTERATION")
counter = 0
while(counter < 1):
  original_sample = sst2_dataset['train'][random.randint(0,len(sst2_dataset['train'])-1)]
  sample = copy.deepcopy(original_sample)
  to_alter = random.randint(0,1)
  if to_alter == 0:
    to_alter = 'hypothesis'
  else:
    to_alter = 'premise'
  altered = hypernym_alteration(sample, to_alter)
  if altered:
    counter += 1
    print("PRE: " + original_sample[to_alter])
    print("POST: " + sample[to_alter])

print("BIRTH DATE QUERY")
counter = 0
while(counter < 1):
  original_sample = sst2_dataset['train'][random.randint(0,len(sst2_dataset['train'])-1)]
  sample = copy.deepcopy(original_sample)
  altered = birth_date_query(sample)
  if altered:
    counter += 1
    print("PRE: " + sample['premise'])
    print("HYP: " + sample['hypothesis'])
    print("LABEL: " + sample['label'])

print("FACT CHECKING QUERY")
counter = 0
while(counter < 1):
  original_sample = sst2_dataset['train'][random.randint(0,len(sst2_dataset['train'])-1)]
  sample = copy.deepcopy(original_sample)
  altered = fact_checking_query(sample)
  if altered:
    counter += 1
    print("PRE: " + sample['premise'])
    print("HYP: " + sample['hypothesis'])
    print("LABEL: " + sample['label'])

print("GROUP INCLUSION QUERY")
counter = 0
while(counter < 1):
  original_sample = sst2_dataset['train'][random.randint(0,len(sst2_dataset['train'])-1)]
  sample = copy.deepcopy(original_sample)
  altered = group_inclusion_query(sample)
  if altered:
    counter += 1
    print("PRE: " + sample['premise'])
    print("HYP: " + sample['hypothesis'])
    print("LABEL: " + sample['label'])

print("RANDOM JOIN")
counter = 0
while(counter < 1):
  original_sample = sst2_dataset['train'][random.randint(0,len(sst2_dataset['train'])-1)]
  other_sample = sst2_dataset['train'][random.randint(0,len(sst2_dataset['train'])-1)]
  sample = copy.deepcopy(original_sample)
  altered = random_join(sample, other_sample)
  if altered:
    counter += 1
    print("PRE: " + sample['premise'])
    print("HYP: " + sample['hypothesis'])
    print("LABEL: " + sample['label'])

SYNONIM ALTERATION
PRE: Martina Hingis ( born 30 September 1980 ) is a Czechoslovak-born Swiss professional tennis player who spent a total of 209 weeks as the singles world No. 1 . She has won five Grand Slam singles titles , twelve Grand Slam women 's doubles titles , winning a calendar-year doubles Grand Slam in 1998 , and five Grand Slam mixed doubles titles ; for a combined total of twenty-two major titles . Hingis set a series of `` youngest-ever '' records during the 1990s , including youngest-ever Grand Slam champion and youngest-ever world No. 1 . Widely considered an all-time tennis great , Hingis was ranked by Tennis magazine in 2005 as the 8th-greatest female player of the preceding 40 years .
POST: Martina Hingis ( born 30 September 1980 ) is a Czechoslovak - born lawn_tennis player who spent a total of 209 weeks as the singles world No . 1 . She has won five Grand Slam singles titles , twelve Grand Slam women 's doubles titles , winning a calendar - year doubles Grand Sla

In [13]:
#Our training dataset has 32958 samples
#Our validation dataset 2288 samples
#I would generate around 3300 augmented samples for training and 300 samples for validation, i.e. 3600. ==> 4600 for introducing more neutral
#Since the dataset is already unbalanced towards entailment and contradiction labels I will not care to much of having equal parts on my adversarial dataset

random.seed(0)

synonym_alteration_samples = 900
hypernym_alteration_samples = 900
birth_date_query_samples = 900
fact_checking_query_samples = 100
group_inclusion_query_samples = 60
mix_one_samples = 20 #Applies fact checking and then synonym alteration on the premise
mix_two_samples = 20 #Applies group inclusion and then synonym alteration on the premise
random_join_samples = 1700

input_dataset = sst2_dataset['train']
adversarial_samples = []

synonym_alteration_counter = 0
hypernym_alteration_counter = 0
birth_date_query_counter = 0
fact_checking_query_counter = 0
group_inclusion_query_counter = 0
mix_one_counter = 0
mix_two_counter = 0
random_join_counter = 0

Let's generate the adversarial samples

In [14]:
while(synonym_alteration_counter < synonym_alteration_samples):
  index = random.randint(0,len(input_dataset)-1)
  sample = copy.deepcopy(sst2_dataset['train'][index])
  to_alter = random.randint(0,1)
  if to_alter == 0:
    to_alter = "premise"
  else:
    to_alter = "hypothesis"
  altered = synonym_alteration(sample, to_alter)
  if altered:
    adversarial_samples.append(sample)
    synonym_alteration_counter += 1

In [15]:
while(hypernym_alteration_counter < hypernym_alteration_samples):
  index = random.randint(0,len(input_dataset)-1)
  sample = copy.deepcopy(sst2_dataset['train'][index])
  to_alter = random.randint(0,1)
  if to_alter == 0:
    to_alter = "premise"
  else:
    to_alter = "hypothesis"
  altered = hypernym_alteration(sample, to_alter)
  if altered:
    adversarial_samples.append(sample)
    hypernym_alteration_counter += 1

In [ ]:
while(birth_date_query_counter < birth_date_query_samples):
  index = random.randint(0,len(input_dataset)-1)
  sample = copy.deepcopy(sst2_dataset['train'][index])
  altered = birth_date_query(sample)
  if altered:
    adversarial_samples.append(sample)
    birth_date_query_counter += 1

In [ ]:
while(fact_checking_query_counter < fact_checking_query_samples):
  index = random.randint(0,len(input_dataset)-1)
  sample = copy.deepcopy(sst2_dataset['train'][index])
  altered = fact_checking_query(sample)
  if altered:
    adversarial_samples.append(sample)
    fact_checking_query_counter += 1

In [ ]:
while(group_inclusion_query_counter < group_inclusion_query_samples):
  index = random.randint(0,len(input_dataset)-1)
  sample = copy.deepcopy(sst2_dataset['train'][index])
  altered = group_inclusion_query(sample)
  if altered:
    adversarial_samples.append(sample)
    group_inclusion_query_counter += 1

In [ ]:
while(mix_one_counter < mix_one_samples):
  index = random.randint(0,len(input_dataset)-1)
  sample = copy.deepcopy(sst2_dataset['train'][index])
  altered = fact_checking_query(sample)
  if altered:
    altered = synonym_alteration(sample, "premise")
    if altered:
      adversarial_samples.append(sample)
      mix_one_counter += 1

In [ ]:
while(mix_two_counter < mix_two_samples):
  index = random.randint(0,len(input_dataset)-1)
  sample = copy.deepcopy(sst2_dataset['train'][index])
  altered = group_inclusion_query(sample)
  if altered:
    altered = synonym_alteration(sample, "premise")
    if altered:
      adversarial_samples.append(sample)
      mix_two_counter += 1

In [ ]:
while(random_join_counter < random_join_samples):
  original_sample = sst2_dataset['train'][random.randint(0,len(sst2_dataset['train'])-1)]
  other_sample = sst2_dataset['train'][random.randint(0,len(sst2_dataset['train'])-1)]
  sample = copy.deepcopy(original_sample)
  random_join(sample, other_sample)
  random_join_counter_counter += 1
  adversarial_samples.append(sample)

In [ ]:
print(len(adversarial_samples))
"""print(adversarial_samples[0])
print(adversarial_samples[5399])"""

In [ ]:
#TO DO: Set saving options

random.seed(0)
random.shuffle(adversarial_samples)
!huggingface-cli login

temp_dict = {}
temp_dict['premise'] = []
temp_dict['hypothesis'] = []
temp_dict['label'] = []
for sample in adversarial_samples: #We remove srl, wsd and id, whose coherence wiht the text is no more guaranteed
  temp_dict['premise'].append(sample['premise'])
  temp_dict['hypothesis'].append(sample['hypothesis'])
  temp_dict['label'].append(sample['label'])

output_dataset = Dataset.from_dict(temp_dict)

output_dataset.push_to_hub("Aivalf/NLI_adversarial_dataset_v2")

